OLD NOTEBOOK: SEE osm_pbf_power_data_extractor.py which does everything.

In [1]:
import os, sys, time
#IMPORTANT: RUN SCRIPT FROM THIS SCRIPTS DIRECTORY i.e data_exploration/ TODO: make more robust
##os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../../scripts')
from iso_country_codes import AFRICA_CC



import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import geoplot
import matplotlib.pyplot as plt
from iso_country_codes import AFRICA_CC
from osm_pbf_power_data_extractor import convert_pd_to_gdf_lines, convert_pd_to_gdf

import logging
logger = logging.getLogger(__name__)

# SUBSTATIONS (Just simple cleaning to test snapping)

### Check old buses

In [2]:
# Load pypsa-eur data
df_all_buses = (pd.read_csv(os.getcwd()+"/entsoegridkit/buses.csv", quotechar="'",
                         true_values='t', false_values='f',
                         dtype=dict(bus_id="str"))
            .set_index("bus_id")
            .drop(['station_id'], axis=1)
            .rename(columns=dict(voltage='v_nom')))

#print(df_all_lines.geometry.unique())
display(df_all_buses)

,v_nom,dc,symbol,under_construction,tags,x,y
bus_id,,,,,,,
1,380.0,False,Substation,False,"""TSO""=>"""", ""oid""=>""29904"", ""country""=>""SA"", ""v...",39.726563,21.665086
2,380.0,False,Substation,False,"""TSO""=>"""", ""oid""=>""29909"", ""country""=>""SA"", ""v...",40.186615,21.851302
3,380.0,False,Substation,False,"""TSO""=>"""", ""oid""=>""29912"", ""country""=>""SA"", ""v...",40.461273,21.719956
4,380.0,False,Substation,False,"""TSO""=>"""", ""oid""=>""29899"", ""country""=>""SA"", ""v...",39.508209,24.718143
5,NaN,True,Substation,False,"""TSO""=>"""", ""oid""=>""29896"", ""country""=>""SA"", ""v...",39.635925,24.776760
...,...,...,...,...,...,...,...
7824,380.0,False,joint,NaN,NaN,27.246094,38.501967
8479,220.0,False,joint,NaN,NaN,15.492096,63.192160
7009,220.0,False,joint,NaN,NaN,1.591644,48.752567


# Import data and create final dataframe layout

In [51]:
#----------- SUBSTATIONS -----------
# Load uncleaned data
df_all_substations = gpd.read_file(os.getcwd()+"/data/raw/africa_all_raw_substations.geojson")

# Modification - create final dataframe layout
df_all_substations = df_all_substations.rename(
    columns = {
        "id": "bus_id",
        "tags.voltage": "voltage",
        # "dc", will be added below
        "tags.power": "symbol",
        # "under_construction", will be added below     
        "tags.substation": "tags_substation",
        "Country": "tags_country",  # new/different to PyPSA-Eur
        "Area": "tags_area",
        "lonlat": "geometry",
    }
)

# Add NaN as default
df_all_substations["dc"] = np.nan
df_all_substations["under_construction"] = np.nan
df_all_substations["x"] = df_all_substations["geometry"].x
df_all_substations["y"] = df_all_substations["geometry"].y

#Rearrange columns
clist = ["bus_id","voltage","dc","symbol","under_construction","tags_substation","tags_country",
         "tags_area","x", "y", "geometry"]
df_all_substations = df_all_substations[clist]

# make float to integer
df_all_substations["bus_id"] = df_all_substations["bus_id"].astype(int)

display(df_all_substations)

,bus_id,voltage,dc,symbol,under_construction,tags_substation,tags_country,tags_area,x,y,geometry
0,6576960395,None,NaN,substation,NaN,None,algeria,NaN,2.771995,36.646372,POINT (2.77199 36.64637)
1,3734017299,None,NaN,substation,NaN,None,algeria,NaN,5.427109,32.699008,POINT (5.42711 32.69901)
2,4526380322,None,NaN,substation,NaN,minor_distribution,algeria,NaN,3.004848,36.728594,POINT (3.00485 36.72859)
3,6573099165,None,NaN,substation,NaN,None,algeria,NaN,2.779837,36.644217,POINT (2.77984 36.64422)
4,3737621021,None,NaN,substation,NaN,minor_distribution,algeria,NaN,3.000195,36.736398,POINT (3.00020 36.73640)
...,...,...,...,...,...,...,...,...,...,...,...
6891,503190678,None,NaN,substation,NaN,None,zimbabwe,5320.0,28.539742,-20.154835,POINT (28.53974 -20.15484)
6892,573691200,None,NaN,substation,NaN,None,zimbabwe,5260.0,30.942052,-17.916373,POINT (30.94205 -17.91637)
6893,179184234,330000;132000,NaN,substation,NaN,transmission,zimbabwe,59230.0,31.376556,-17.309747,POINT (31.37656 -17.30975)
6894,451713273,None,NaN,substation,NaN,None,zimbabwe,2340.0,32.627628,-18.985457,POINT (32.62763 -18.98546)


### Define under_construction, dc, filter "transmission"

In [52]:
df_all_substations["under_construction"] = True
df_all_substations["dc"] = False
df_all_substations = df_all_substations[df_all_substations["tags_substation"] == "transmission"] # keep only rows with indexed "transmission"
display(df_all_substations)

,bus_id,voltage,dc,symbol,under_construction,tags_substation,tags_country,tags_area,x,y,geometry
52,221085150,400000;220000,False,substation,True,transmission,algeria,241710.0,3.574875,36.250907,POINT (3.57488 36.25091)
54,377450818,None,False,substation,True,transmission,algeria,69840.0,1.858983,35.628363,POINT (1.85898 35.62836)
73,381266492,60000,False,substation,True,transmission,algeria,18850.0,-0.915273,35.572792,POINT (-0.91527 35.57279)
83,173670759,None,False,substation,True,transmission,algeria,265820.0,2.601559,36.450328,POINT (2.60156 36.45033)
90,653738025,None,False,substation,True,transmission,algeria,1910.0,-0.633250,35.219708,POINT (-0.63325 35.21971)
...,...,...,...,...,...,...,...,...,...,...,...
6880,393492355,88000;11000,False,substation,True,transmission,zimbabwe,4960.0,28.481795,-20.158511,POINT (28.48179 -20.15851)
6886,240035191,132000,False,substation,True,transmission,zimbabwe,7830.0,30.940673,-17.782664,POINT (30.94067 -17.78266)
6889,409152931,132000,False,substation,True,transmission,zimbabwe,11120.0,31.085101,-17.748434,POINT (31.08510 -17.74843)
6893,179184234,330000;132000,False,substation,True,transmission,zimbabwe,59230.0,31.376556,-17.309747,POINT (31.37656 -17.30975)


In [53]:
# df_all_substations.tags_area.count()

In [54]:
# df_all_substations[df_all_substations["tags_substation"] == "transmission"].tags_area.describe()

In [55]:
# df_all_substations[df_all_substations["tags_substation"] == "distribution"].tags_area.describe()

In [56]:
# df_all_substations[df_all_substations["tags_substation"] == "industrial"].tags_area.describe()

In [57]:
# df_all_substations[df_all_substations["tags_substation"].isna()].tags_area.describe()

In [58]:
# df_all_substations["tags_substation"].unique()

### Clean voltage

In [59]:
# Drop any row with Voltage = N/A
df = df_all_substations.dropna(subset=['voltage']) 

#Split semicolon separated cells i.e. [66000;220000] and create new identical rows
lst_col = 'voltage'
x = df.assign(**{lst_col:df[lst_col].str.split(';')})
x = pd.DataFrame({
    col:np.repeat(x[col].values, x[lst_col].str.len())
    for col in x.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]
df_all_substations = x

#display(df_all_substations)

In [60]:
# Create unique bus id's
# The steps below create unique bus id's without loosing the original OSM bus_id 

# Context
# The previous duplication of rows (to split the voltage) lead to a couple of same bus_id

# Method
# Unique bus_id are created by simply adding -1,-2,-3 to the original bus_id
# Every unique id gets a -1 
# If a bus_id exist i.e. three times it it will the counted by cumcount -1,-2,-3 making the id unique

if df_all_substations["bus_id"].count() != df_all_substations["bus_id"].nunique(): # operate only if line_id is not already unique (nunique counts unique values)
    df_all_substations["cumcount"] = df_all_substations.groupby(["bus_id"]).cumcount() # create cumcount column. Cumcount counts 0,1,2,3 the number of duplicates
    df_all_substations["cumcount"] = df_all_substations["cumcount"] + 1 # avoid 0 value for better understanding
    df_all_substations["bus_id"] = df_all_substations["bus_id"].astype(str) + "-" + df_all_substations["cumcount"].values.astype(str) # add cumcount to line_id to make line_id unique
    df_all_substations.drop(columns = "cumcount", inplace=True) # remove cumcount column

display(df_all_substations)

,bus_id,voltage,dc,symbol,under_construction,tags_substation,tags_country,tags_area,x,y,geometry
0,221085150-1,400000,False,substation,True,transmission,algeria,241710.0,3.574875,36.250907,POINT (3.57488 36.25091)
1,221085150-2,220000,False,substation,True,transmission,algeria,241710.0,3.574875,36.250907,POINT (3.57488 36.25091)
2,381266492-1,60000,False,substation,True,transmission,algeria,18850.0,-0.915273,35.572792,POINT (-0.91527 35.57279)
3,751040071-1,400000,False,substation,True,transmission,algeria,120930.0,-0.168282,35.766294,POINT (-0.16828 35.76629)
4,751040071-2,220000,False,substation,True,transmission,algeria,120930.0,-0.168282,35.766294,POINT (-0.16828 35.76629)
...,...,...,...,...,...,...,...,...,...,...,...
1857,393492355-2,11000,False,substation,True,transmission,zimbabwe,4960.0,28.481795,-20.158511,POINT (28.48179 -20.15851)
1858,240035191-1,132000,False,substation,True,transmission,zimbabwe,7830.0,30.940673,-17.782664,POINT (30.94067 -17.78266)
1859,409152931-1,132000,False,substation,True,transmission,zimbabwe,11120.0,31.085101,-17.748434,POINT (31.08510 -17.74843)
1860,179184234-1,330000,False,substation,True,transmission,zimbabwe,59230.0,31.376556,-17.309747,POINT (31.37656 -17.30975)


In [61]:
# Remove all non-numeric values

df_all_substations['voltage'] = df_all_substations['voltage'].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(float) # if cell can't converted to float -> nan
df_all_substations = df_all_substations.dropna(subset=['voltage']) # Drop any row with Voltage = N/A
df_all_substations.loc[:,"voltage"]  = df_all_substations['voltage'].astype(int)
#df_all_lines['voltage'].unique()

In [62]:
# Keep rows with x > 110 kV as it is considered as transmission level

df_all_substations = df_all_substations[df_all_substations.voltage > 110000]

In [63]:
# display(df_all_substations)
# display(df_all_substations['voltage'].unique())

In [67]:
## Generate Files (CSV+GeoJSON) 

# #### CSV
# outputfile_partial = os.path.join(os.getcwd(), "data", "clean", "africa_all" + "_buses" + "_clean") # Output file directory

# if not os.path.exists(outputfile_partial):
#     os.makedirs(os.path.dirname(outputfile_partial), exist_ok=True) #  create clean directoryif not already exist
    
# df_all_substations.to_csv(outputfile_partial + "_buses"+ ".csv")  # Generate CSV


# #### GEOJSON

# df_all_substations = gpd.GeoDataFrame(df_all_substations, geometry="geometry",crs="EPSG:4326")
# df_all_substations.to_file(outputfile_partial + "_buses"+ ".geojson", driver="GeoJSON")    


# LINES 

### Check old unique values

In [68]:
# Load pypsa-eur data
df_all_lines = (pd.read_csv(os.getcwd()+"/entsoegridkit/lines.csv", quotechar="'", true_values='t', false_values='f',
                         dtype=dict(line_id='str', bus0='str', bus1='str',
                                    underground="bool", under_construction="bool")).set_index('line_id').rename(columns=dict(voltage='v_nom', circuits='num_parallel')))

#print(df_all_lines.geometry.unique())
display(df_all_lines)

,bus0,bus1,v_nom,num_parallel,length,underground,under_construction,tags,geometry
line_id,,,,,,,,,
6568,33,34,132,1,25687.871153,False,False,"""MW""=>""0.0"", ""TSO""=>"" "", ""oid""=>""51000"", ""EIC_...","LINESTRING(32.2819899090672 26.2326704525422,3..."
11675,31,34,132,1,88419.785937,False,False,NaN,"LINESTRING(32.3037477094187 26.0016233689967,3..."
6571,37,40,132,1,26394.698842,False,False,"""MW""=>""0.0"", ""TSO""=>"" "", ""oid""=>""51003"", ""EIC_...","LINESTRING(32.6501207538248 26.344729862945,32..."
6569,34,40,132,1,43972.508007,False,False,"""MW""=>""0.0"", ""TSO""=>"" "", ""oid""=>""51001"", ""EIC_...","LINESTRING(32.7250972033021 26.1158696376196,3..."
6570,40,42,132,1,23509.128839,False,False,"""MW""=>""0.0"", ""TSO""=>"" "", ""oid""=>""51002"", ""EIC_...","LINESTRING(32.7259342990352 26.1158221410256,3..."
...,...,...,...,...,...,...,...,...,...
12574,6885,8002,750,1,149251.722487,False,False,NaN,"LINESTRING(30.8701295047876 47.1281577300625,3..."
11495,3065,8002,750,1,49796.609048,False,True,NaN,"LINESTRING(30.2608839547144 47.295882632508,30..."
12573,8002,8424,750,1,90661.434110,False,False,NaN,"LINESTRING(29.956055 46.6076264806737,29.95971..."


# Import data and create final dataframe layout

### Load cables

In [69]:
# Load raw cable data
df_cables = gpd.read_file(os.getcwd()+"/data/raw/africa_all_raw_cables.geojson") 

# Modification - create final dataframe layout
df_cables = df_cables.rename(
    columns = {
        "id": "line_id",
        "tags.voltage": "voltage",
        "tags.circuits": "circuits",
        "tags.cables": "cables",
        "tags.frequency": "tag_frequency",
        "tags.power": "tag_type",
        "tags.location": "tag_location",
        "lonlat": "geometry",
        "Country": "country",  # new/different to PyPSA-Eur
        "Length": "length",
    }
)

# Add NaN as default
df_cables["bus0"] = np.nan
df_cables["bus1"] = np.nan
#df_all_cables["length"] = np.nan # Now in dataset
df_cables["underground"] = np.nan
df_cables["under_construction"] = np.nan

#Rearrange columns
clist = ["line_id","bus0","bus1","voltage","circuits","length","underground",
         "under_construction","tag_type","tag_frequency", "tag_location","geometry", "country"]
df_cables = df_cables[clist]

# make float to integer
df_cables["line_id"] = df_cables["line_id"].astype(int)


#display(df_cables)
#df_all_cables[df_all_cables['tag_location']== "overground"]
#df_all_cables["tags.location"].unique()

### Load lines

In [70]:
# Load raw line data
df_lines = gpd.read_file(os.getcwd()+"/data/raw/africa_all_raw_lines.geojson") 

# Modification - create final dataframe layout
df_lines = df_lines.rename(
    columns = {
        "id": "line_id",
        "tags.voltage": "voltage",
        "tags.circuits": "circuits",
        "tags.cables": "cables",
        "tags.frequency": "tag_frequency",
        "tags.power": "tag_type",
        "lonlat": "geometry",
        "Country": "country",  # new/different to PyPSA-Eur
        "Length": "length",
    }
)

# Add NaN as default
df_lines["bus0"] = np.nan
df_lines["bus1"] = np.nan
#df_all_lines["length"] = np.nan # commented because, we have now length data
df_lines["underground"] = np.nan
df_lines["under_construction"] = np.nan

#Rearrange columns
clist = ["line_id","bus0","bus1","voltage","circuits","length","underground",
         "under_construction","tag_type","tag_frequency", "cables","geometry", "country"]
df_lines = df_lines[clist]

#display(df_all_lines)

### Combine cable and line to one  "df_all_lines" dataset

In [71]:
df_all_lines = pd.concat([df_lines,df_cables])
# df_all_lines

### Define underground, under_construction information, frequency, circuits

In [72]:
# under construction
df_all_lines["under_construction"] = False # default. Not more information atm available

# underground
df_all_lines["underground"] = (df_all_lines["tag_type"] == "cable") # Simplified. If tag_type cable then underground is True. 
# More information extractable for "underground" by looking at "tag_location".
if 'tag_location' in df_all_lines: # drop column if exist
    df_all_lines.drop(columns = "tag_location", inplace=True)

# frequency
df_all_lines["tag_frequency"] = 50
#df_all_lines["tag_frequency"].unique()

# circuits
if df_all_lines["cables"].dtype != int: # if not int make int
    df_all_lines.loc[(df_all_lines["cables"] < "3") | df_all_lines["cables"].isna(), "cables"] = "0" #HERE. "0" if cables "None", "nan" or "1"
    df_all_lines["cables"] = df_all_lines["cables"].astype("int")
if 4 or 5 in df_all_lines["cables"].values: # downgrade 4 and 5 cables to 3... 
    # Reason: 4 cables have 1 lighting protection cables, 5 cables has 2 LP cables - not transferring energy; 
    # see https://hackaday.com/2019/06/11/a-field-guide-to-transmission-lines/
    df_all_lines.loc[(df_all_lines["cables"] == 4) | (df_all_lines["cables"] == 5), "cables"] = 3 # where circuits are "0" make "1"
df_all_lines.loc[df_all_lines["circuits"].isna(), "circuits"] = df_all_lines.loc[df_all_lines['circuits'].isna(), "cables"] / 3 # one circuit contains 3 cables
df_all_lines["circuits"] = df_all_lines["circuits"].astype(int)
df_all_lines.loc[(df_all_lines["circuits"] == "0") | (df_all_lines["circuits"] == 0), "circuits"] = 1 # where circuits are "0" make "1"

if 'cables' in df_all_lines: # drop column if exist
    df_all_lines.drop(columns = "cables", inplace=True)

# df_all_lines["circuits"].unique()
# df_all_lines["cables"].unique()
# display(df_all_lines)

### Clean voltage

In [73]:
# Drop any row with Voltage = N/A
df = df_all_lines.dropna(subset=['voltage']) 

#Split semicolon separated cells i.e. [66000;220000] and create new identical rows
lst_col = 'voltage'
x = df.assign(**{lst_col:df[lst_col].str.split(';')})
x = pd.DataFrame({
    col:np.repeat(x[col].values, x[lst_col].str.len())
    for col in x.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]
df_all_lines = x

#display(df_all_lines)

In [74]:
# Create unique line_id's
# The steps below create unique line_id's without loosing the original OSM line_id 

# Context
# The previous duplication of rows (to split the voltage) lead to a couple of same line_id (about 30% of dataset)

# Method
# Unique line_id are created by simply adding -1,-2,-3 to the original line_id
# Every unique id gets a -1 
# If a line_id exist i.e. three times it it will the counted by cumcount -1,-2,-3 making the id unique

if df_all_lines["line_id"].count() != df_all_lines["line_id"].nunique(): # operate only if line_id is not already unique (nunique counts unique values)
    df_all_lines["cumcount"] = df_all_lines.groupby(["line_id"]).cumcount() # create cumcount column. Cumcount counts 0,1,2,3 the number of duplicates
    df_all_lines["cumcount"] = df_all_lines["cumcount"] + 1 # avoid 0 value for better understanding
    df_all_lines["line_id"] = df_all_lines["line_id"].astype(str) + "-" + df_all_lines["cumcount"].values.astype(str) # add cumcount to line_id to make line_id unique
    df_all_lines.drop(columns = "cumcount", inplace=True) # remove cumcount column

#display(df_all_lines)

In [75]:
# Remove all non-numeric values

df_all_lines.loc[:,"voltage"] = df_all_lines['voltage'].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(float) # if cell can't converted to float -> nan
df_all_lines = df_all_lines.dropna(subset=['voltage']) # Drop any row with Voltage = N/A
df_all_lines.loc[:,"voltage"]  = df_all_lines['voltage'].astype(int)
#df_all_lines['voltage'].unique()

/home/max/anaconda3/envs/pypsa-africa/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [76]:
# Keep rows with x > 110 kV as it is considered as transmission level
df_all_lines = df_all_lines[df_all_lines.voltage > 110000]

# Remove lines that are shorter than 100m
#df_all_lines = df_all_lines[df_all_lines.length > 100]

In [77]:
display(df_all_lines)
display(df_all_lines['voltage'].unique())
display(df_all_lines['length'].describe())

,line_id,bus0,bus1,voltage,circuits,length,underground,under_construction,tag_type,tag_frequency,geometry,country
0,380986751-1,NaN,NaN,220000,1,40467.295973,False,False,line,50,"LINESTRING (-1.83638 34.78691, -1.83590 34.789...",algeria
1,173682341-1,NaN,NaN,400000,1,168.067783,False,False,line,50,"LINESTRING (-0.90176 35.08983, -0.90181 35.090...",algeria
2,655093026-1,NaN,NaN,220000,1,64.987785,False,False,line,50,"LINESTRING (0.14621 34.90905, 0.14639 34.90922...",algeria
5,383315204-1,NaN,NaN,220000,1,98637.025052,False,False,line,50,"LINESTRING (3.38017 36.71034, 3.38385 36.71236...",algeria
8,655093020-1,NaN,NaN,220000,1,110.799435,False,False,line,50,"LINESTRING (0.14661 34.90939, 0.14672 34.90930...",algeria
...,...,...,...,...,...,...,...,...,...,...,...,...
15178,456151088-1,NaN,NaN,132000,1,131.900676,True,False,cable,50,"LINESTRING (31.01196 -29.81361, 31.01196 -29.8...",south-africa
15179,668505779-1,NaN,NaN,132000,1,36903.154434,True,False,cable,50,"LINESTRING (39.19950 -6.63733, 39.19991 -6.637...",tanzania
15180,445315079-1,NaN,NaN,150000,1,948.419880,True,False,cable,50,"LINESTRING (9.97671 37.21045, 9.96819 37.21059)",tunisia
15263,696726292-1,NaN,NaN,132000,1,155.359211,True,False,cable,50,"LINESTRING (33.13385 0.50202, 33.13392 0.50195...",uganda


array([220000, 400000, 225000, 150000, 132000, 330000, 350000, 161000,
       120000, 500000, 230000, 115000, 138000, 533000, 275000, 765000,
       380000])

count    9.948000e+03
mean     2.605750e+04
std      5.726067e+04
min      3.055504e+00
25%      1.109375e+02
50%      1.569921e+03
75%      2.538909e+04
max      1.005068e+06
Name: length, dtype: float64

In [80]:
## Generate Files (CSV+GeoJSON) 


# ### CSV
# outputfile_partial = os.path.join(os.getcwd(), "data", "clean", "africa_all" + "_lines" + "_clean") # Output file directory

# if not os.path.exists(outputfile_partial):
#     os.makedirs(os.path.dirname(outputfile_partial), exist_ok=True) #  create clean directoryif not already exist
    
# df_all_lines.to_csv(outputfile_partial + "_lines"+ ".csv")  # Generate CSV


# ### GEOJSON
# df_all_lines = gpd.GeoDataFrame(df_all_lines, geometry="geometry",crs="EPSG:4326")
# df_all_lines.to_file(outputfile_partial + "_lines"+ ".geojson", driver="GeoJSON")    
  


# GENERATORS

In [ ]:
# ----------- Generator -----------

#Load uncleaned data
df_all_generators = pd.read_csv(os.getcwd()+"/data/raw/africa_all_raw_generators.csv")

#Clean data
df_all_generators = df_all_generators.reset_index(drop=True)
df_all_generators = df_all_generators[df_all_generators['tags.generator:output:electricity'].astype(str).str.contains('MW')] #removes boolean 
df_all_generators['tags.generator:output:electricity'] = df_all_generators['tags.generator:output:electricity'].str.extract('(\d+)').astype(float)
df_all_generators = df_all_generators.rename(columns = {'tags.generator:output:electricity':"power_output_MW"})


## Generate Files

#CSV
#outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_generators'+'_cleaned.')
#df_all_generators.to_csv(outputfile_partial + 'csv') # Generate CSV

#GeoJSON
# gdf_generators = convert_pd_to_gdf(df_all_generators)
# gdf_generators.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


#display(df_all_generators)